In [20]:
#### This script is to aggregate functional group layers up to biome level using data from Global Ecosystem Typology https://global-ecosystems.org/explore
## rename files to replace . with _ as the code below fails otherwise

import os

# Define the path to the target folder
folder_path = "O:/PP2024/ecoregion_test/GET"

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a .tif file
    if filename.lower().endswith('.tif'):
        # Generate the new filename by replacing periods with underscores
        # Only replace periods in the filename part, not the extension
        name, ext = os.path.splitext(filename)
        new_name = name.replace('.', '_') + ext
        
        # Construct the full file paths
        original_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, new_name)
        
        # Rename the file
        os.rename(original_file_path, new_file_path)
        
        print(f"Renamed file: {original_file_path} to {new_file_path}")

print("All .tif files have been processed.")


Renamed file: O:/PP2024/ecoregion_test/GET\F1.1.web.mix_v2.0.tif to O:/PP2024/ecoregion_test/GET\F1_1_web_mix_v2_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F1.2.web.map_v1.0.tif to O:/PP2024/ecoregion_test/GET\F1_2_web_map_v1_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F1.3.web.map_v1.0.tif to O:/PP2024/ecoregion_test/GET\F1_3_web_map_v1_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F1.4.web.map_v1.0.tif to O:/PP2024/ecoregion_test/GET\F1_4_web_map_v1_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F1.5.web.map_v1.0.tif to O:/PP2024/ecoregion_test/GET\F1_5_web_map_v1_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F1.6.web.mix_v1.0.tif to O:/PP2024/ecoregion_test/GET\F1_6_web_mix_v1_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F1.7.web.orig_v2.0.tif to O:/PP2024/ecoregion_test/GET\F1_7_web_orig_v2_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F2.1.web.alt_v4.0.tif to O:/PP2024/ecoregion_test/GET\F2_1_web_alt_v4_0.tif
Renamed file: O:/PP2024/ecoregion_test/GET\F2.

In [21]:
### reclassify GET rasters - keeping major occurances and removing everything else

import arcpy
from arcpy.sa import *

# Set the environment settings
arcpy.env.workspace = "O:/PP2024/ecoregion_test/GET"
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# Define the output directory
output_directory = "O:/PP2024/ecoregion_test/GET/reclass"

# List all rasters in the workspace
rasters = arcpy.ListRasters()

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Define the reclassification rules
remap = RemapValue([[1, 1], [2, 0], [3, 0], ["NODATA", 0]])

# Process each raster
for raster in rasters:
    # Perform the reclassification
    reclassified_raster = Reclassify(raster, "Value", remap, "NODATA")
    
    # Define the output path
    output_raster = f"{output_directory}/{raster}"
    
    # Save the reclassified raster
    reclassified_raster.save(output_raster)
    
    print(f"Reclassified raster saved as {output_raster}")

# Release the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")

print("Reclassification completed for all rasters.")

Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_1_web_mix_v2_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_2_web_map_v1_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_3_web_map_v1_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_4_web_map_v1_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_5_web_map_v1_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_6_web_mix_v1_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F1_7_web_orig_v2_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F2_10_web_mix_v2_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F2_1_web_alt_v4_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F2_2_web_mix_v2_0.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/F2_3_web_mix_v2_0.tif
Reclassified raster saved as O

In [22]:
## project a single raster so all others can be matched to it

# raster to project
target = "O:/PP2024/ecoregion_test/GET/reclass/F1_1_web_mix_v2_0.tif"

# coordinate system to project to
sr = arcpy.SpatialReference("Mollweide (world)")

# project raster
arcpy.management.ProjectRaster(in_raster = target, 
                               out_raster = "O:/PP2024/ecoregion_test/GET/reclass/moll/moll_F1_1_web_mix_v2_0.tif",
                               out_coor_system = sr, 
                               resampling_type = "NEAREST")

<Result 'O:\\PP2024\\ecoregion_test\\GET\\reclass\\moll\\moll_F1_1_web_mix_v2_0.tif'>

In [23]:
### ## project other rasters to match the one above
import arcpy
import os

# Set workspace and output directory
workspace = r"O:/PP2024/ecoregion_test/GET/reclass"
output_folder = r"O:/PP2024/ecoregion_test/GET/reclass/moll"
arcpy.env.workspace = workspace


# List all rasters in the workspace
rasters = arcpy.ListRasters()

# Define the target projection (Mollweide)
mollweide_proj = arcpy.SpatialReference("Mollweide (world)")

# Determine the target cell size (use the cell size of the first raster as reference)
reference_raster = "O:/PP2024/ecoregion_test/GET/reclass/moll/moll_F1_1_web_mix_v2_0.tif"
desc = arcpy.Describe(reference_raster)
target_cell_size = desc.meanCellWidth  # or you can specify your own cell size

# Align rasters by snapping to the lower-left corner of the reference raster
snap_raster = os.path.join(output_folder, "snap_raster.tif")
arcpy.management.ProjectRaster(reference_raster, snap_raster, mollweide_proj, 
                               cell_size=target_cell_size)
arcpy.env.snapRaster = snap_raster

# Reproject all rasters
for raster in rasters:
    # Define the output path
    output_raster = os.path.join(output_folder, os.path.basename(raster))
    
    # Reproject the raster
    arcpy.management.ProjectRaster(
        in_raster=raster,
        out_raster=output_raster,
        out_coor_system=mollweide_proj,
        resampling_type="NEAREST",  # or BILINEAR/CUBIC if preferred
        cell_size=target_cell_size,
        geographic_transform=""
    )

print("All rasters have been projected and aligned.")

arcpy.env.snapRaster = None
arcpy.management.Delete("O:/PP2024/ecoregion_test/GET/reclass/moll/moll_F1_1_web_mix_v2_0.tif")
arcpy.management.Delete("O:/PP2024/ecoregion_test/GET/reclass/moll/snap_raster.tif")
print("snap layers deleted")

All rasters have been projected and aligned.
snap layers deleted


In [24]:
## sum rasters together based on prefix which identifies the biome 
import arcpy
from arcpy.sa import *

# Set the environment settings
arcpy.env.workspace = r"O:/PP2024/ecoregion_test/GET/reclass/moll"
arcpy.env.addOutputsToMap = False

# List all rasters in the workspace
rasters = arcpy.ListRasters()

# Create a dictionary to store the lists of rasters by their prefixes
raster_groups = {}

# Group rasters by their prefixes
for raster in rasters:
    prefix = raster.split('_')[0]
    if prefix not in raster_groups:
        raster_groups[prefix] = []
    raster_groups[prefix].append(raster)

# Sum the rasters in each group
for prefix, raster_list in raster_groups.items():
    # Initialize the sum with the first raster
    raster_sum = Raster(raster_list[0])
    
    # Add the rest of the rasters in the group
    for raster in raster_list[1:]:
        raster_sum += Raster(raster)
    
    # Save the resulting summed raster
    output_raster = f"O:/PP2024/ecoregion_test/GET/reclass/moll/sum/{prefix}_sum.tif"
    raster_sum.save(output_raster)
    
    print(f"Summed rasters for prefix {prefix} saved as {output_raster}")

print("All raster summations completed.")


Summed rasters for prefix F1 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/F1_sum.tif
Summed rasters for prefix F2 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/F2_sum.tif
Summed rasters for prefix F3 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/F3_sum.tif
Summed rasters for prefix FM1 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/FM1_sum.tif
Summed rasters for prefix M1 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/M1_sum.tif
Summed rasters for prefix M2 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/M2_sum.tif
Summed rasters for prefix M3 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/M3_sum.tif
Summed rasters for prefix M4 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/M4_sum.tif
Summed rasters for prefix MFT1 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/MFT1_sum.tif
Summed rasters for prefix MT1 saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/MT1_sum.tif
Summed rasters for prefix MT2 saved as O:/

In [25]:
### reclassify GET rasters again to 1 and 0

import arcpy
from arcpy.sa import *

# Set the environment settings
arcpy.env.workspace = "O:/PP2024/ecoregion_test/GET/reclass/moll/sum"
arcpy.env.overwriteOutput = True

# Define the output directory
output_directory = "O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass"

# List all rasters in the workspace
rasters = arcpy.ListRasters()

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Define the reclassification rules
remap = RemapValue([[0, 0], [1, 1], [2, 1], [3, 1], [4, 1],[5, 1],[6, 1],[7, 1],[8, 1], ["NODATA", 0]])

# Process each raster
for raster in rasters:
    # Perform the reclassification
    reclassified_raster = Reclassify(raster, "Value", remap, "NODATA")
    
    # Define the output path
    output_raster = f"{output_directory}/{raster}"
    
    # Save the reclassified raster
    reclassified_raster.save(output_raster)
    
    print(f"Reclassified raster saved as {output_raster}")

# Release the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")

print("Reclassification completed for all rasters.")

Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/F1_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/F2_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/F3_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/FM1_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/M1_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/M2_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/M3_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/M4_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/MFT1_sum.tif
Reclassified raster saved as O:/PP2024/ecoregion_test/GET/reclass/moll/sum/sum_reclass/MT1_sum.tif
Reclassified ras